In [36]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [79]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import os
    
def get_corp_code():
    url = "http://comp.fnguide.com/XML/Market/CompanyList.txt"
    
    '''requests.get(url) is a Python function call using 
    the requestslibrary to perform an HTTP GET request 
    to the specified URL. This function is commonly used 
    in web scraping and API interactions to retrieve data 
    from a web server.'''    
    resp = requests.get(url)
    
    #encoding of the HTTP response content. 
    resp.encoding = "utf-8-sig"
    '''Parse the JSON (JavaScript Object Notation) data from the response
    JSON data is structured, easy to read, and commonly used for exchanging 
    information between different applications or systems, especially 
    in web development and APIs.'''
    data = resp.json()
    comp = data['Co']
    df = pd.DataFrame(data=comp)
    '''gb==701(stock) / 752(ritz)/ 770(etf)/ 780(future)'''
    cond = df['gb'] == '701'
    df2 = df[cond].copy()
    return df2


In [81]:
def get_closing_accounts_day(code):
    url = f"https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{code}"
    selector = "#compBody > div.section.ul_corpinfo > div.corp_group1 > h2:nth-child(4)"
    resp = requests.get(url)
    html = resp.text 
#to create a BeautifulSoup object to parse an HTML document 
# using the lxml parser
    soup = BeautifulSoup(html, "lxml")
    tags = soup.select(selector)
    return tags[0].text

In [82]:
def get_roic(code):
    url = f"http://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701"
    try:
        dfs = pd.read_html(url)
        df = dfs[0]
        df.set_index(df.columns[0], inplace=True)
        '''column : regex(Regular Expression)="^2022" : begins with "2022". 
        row : axis=0 is 'ROIC' '''
        roic = df.filter(regex="^2022").filter(like='ROIC', axis=0).iloc[0, 0]
        roic = float(roic)
    except:
        roic = np.nan
    return roic 

In [83]:
def get_ev_ebitda(code):
    url = f"http://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701"
    try:
        dfs = pd.read_html(url)
        df = dfs[3]
        df.set_index(df.columns[0], inplace=True)
        ev_ebitda = df.filter(like="EV/EBITDA", axis=0).filter(regex="^2022").iloc[0, 0]
        ev_ebitda = float(ev_ebitda)
    except:
        ev_ebitda = np.nan
    return ev_ebitda

In [78]:
if not os.path.exists(f"./data/scraping data"):
    os.makedirs(f"./data/scraping data")

if __name__ == "__main__":
    df = get_corp_code()
    columns = ['code', 'name', 'day', 'roic', 'ev/ebitda']
    data = []
    for i in range(len(df)):
#ex > repeatation of this {"cd":"A157500", "nm":"TIGER 증권", "gb":"770"}
        s = df.iloc[i]
        acode = s['cd']
        code = acode[1:]
        name = s['nm']

        day = get_closing_accounts_day(code)
        roic = get_roic(code) 
        ev_ebitda = get_ev_ebitda(code) 
        data.append((code, name, day, roic, ev_ebitda))
        print(i, code, name, day, roic, ev_ebitda)

        if (i+1) % 100 == 0:
            df1 = pd.DataFrame(data=data, columns=columns)
            df1.to_excel(f"./data/scraping data/data_{i+1}.xlsx")
            data = []

        time.sleep(0.000000001)

    # last companies            
    df1 = pd.DataFrame(data=data, columns=columns)
    df1.to_excel(f"./data/scraping data/data_last.xlsx")

0 060310 3S 03월 결산 3.7 42.75
1 095570 AJ네트웍스 12월 결산 6.1 4.53
2 006840 AK홀딩스 12월 결산 -0.8 11.39
3 054620 APS 12월 결산 -39.6 nan
4 265520 AP시스템 12월 결산 52.4 2.88
5 211270 AP위성 12월 결산 12.7 30.09
6 027410 BGF 12월 결산 19.7 6.72
7 282330 BGF리테일 12월 결산 17.9 5.77
8 126600 BGF에코머티리얼즈 12월 결산 8.5 7.37
9 138930 BNK금융지주 12월 결산 nan nan
10 001460 BYC 12월 결산 nan 6.99
11 013720 CBI 12월 결산 1.4 57.28
12 001040 CJ 12월 결산 5.2 5.62
13 079160 CJ CGV 12월 결산 -2.5 12.64
14 035760 CJ ENM 12월 결산 2.1 3.73
15 311690 CJ 바이오사이언스 12월 결산 -583.4 nan
16 000120 CJ대한통운 12월 결산 4.9 5.65
17 011150 CJ씨푸드 12월 결산 10.5 11.2
18 097950 CJ제일제당 12월 결산 5.6 6.54
19 051500 CJ프레시웨이 12월 결산 14.4 5.14
20 058820 CMG제약 12월 결산 1.2 56.71
21 023460 CNH 12월 결산 nan nan
22 056730 CNT85 12월 결산 11.6 54.3
23 000480 CR홀딩스 12월 결산 4.1 15.42
24 065770 CS 12월 결산 -55.6 nan
25 083660 CSA 코스믹 12월 결산 -73.2 nan
26 000590 CS홀딩스 12월 결산 12.5 6.51
27 012030 DB 12월 결산 18.8 5.95
28 404950 DB금융스팩10호 12월 결산 nan nan
29 456440 DB금융스팩11호 12월 결산 nan nan
30 367360 DB금융스팩9호 12월 결

244 064960 SNT모티브 12월 결산 17.0 4.29
245 100840 SNT에너지 12월 결산 2.2 21.95
246 036530 SNT홀딩스 12월 결산 11.1 5.83
247 005610 SPC삼립 12월 결산 8.3 6.28
248 011810 STX 12월 결산 -0.4 42.15
249 077970 STX엔진 12월 결산 -2.3 81.72
250 071970 STX중공업 12월 결산 4.2 19.73
251 002820 SUN&L 12월 결산 -5.5 1008.09
252 289080 SV인베스트먼트 03월 결산 nan nan
253 084870 TBH글로벌 12월 결산 1.3 6.61
254 002710 TCC스틸 12월 결산 18.0 7.39
255 089230 THE E&M 12월 결산 -3.3 26.83
256 161570 THE MIDONG 12월 결산 -9.7 nan
257 032540 TJ미디어 12월 결산 8.4 13.63
258 022220 TKG애강 12월 결산 11.0 11.7
259 069260 TKG휴켐스 12월 결산 28.4 5.67
260 048770 TPC 12월 결산 2.0 28.36
261 246690 TS인베스트먼트 12월 결산 nan nan
262 317240 TS트릴리온 12월 결산 -9.2 nan
263 002900 TYM 12월 결산 21.6 4.53
264 024070 WISCOM 12월 결산 1.0 8.75
265 057030 YBM넷 12월 결산 47.9 5.94
266 037270 YG PLUS 12월 결산 30.2 32.34
267 040300 YTN 12월 결산 141.3 22.5
268 051390 YW 12월 결산 7.1 10.6
269 052220 iMBC 12월 결산 18.0 16.19
270 079940 가비아 12월 결산 22.7 4.01
271 078890 가온그룹 12월 결산 3.9 11.01
272 000500 가온전선 12월 결산 6.0 5.35
273 399720

491 084690 대상홀딩스 12월 결산 4.6 7.22
492 036480 대성미생물 12월 결산 -0.5 20.31
493 128820 대성산업 12월 결산 5.0 9.08
494 117580 대성에너지 12월 결산 1.0 11.3
495 027830 대성창투 12월 결산 nan nan
496 104040 대성파인텍 12월 결산 -3.7 1256.82
497 129920 대성하이텍 12월 결산 5.7 11.18
498 016710 대성홀딩스 12월 결산 0.6 47.5
499 438220 대신밸런스제13호스팩 12월 결산 nan nan
500 442310 대신밸런스제14호스팩 12월 결산 nan nan
501 457390 대신밸런스제15호스팩 월 결산 nan nan
502 457630 대신밸런스제16호스팩 월 결산 nan nan
503 020180 대신정보통신 03월 결산 21.9 21.2
504 003540 대신증권 12월 결산 nan nan
505 045390 대아티아이 12월 결산 52.2 20.44
506 009190 대양금속 12월 결산 8.2 15.72
507 108380 대양전기공업 12월 결산 -5.3 nan
508 006580 대양제지 12월 결산 -3.7 47.91
509 252980 대영씨엔이 12월 결산 13.3 6.76
510 014160 대영포장 12월 결산 6.1 8.23
511 047040 대우건설 12월 결산 24.3 2.17
512 009320 대우부품 12월 결산 -9.9 nan
513 140320 대우산업개발 12월 결산 -36.7 nan
514 003090 대웅 12월 결산 13.8 6.62
515 069620 대웅제약 12월 결산 11.2 16.79
516 007680 대원 12월 결산 6.0 5.41
517 000430 대원강업 12월 결산 7.1 6.42
518 048910 대원미디어 12월 결산 13.0 6.52
519 005710 대원산업 12월 결산 12.7 2.07
520 006340 대원전선 12월 결산

737 004990 롯데지주 12월 결산 4.8 9.59
738 005300 롯데칠성 12월 결산 7.8 7.5
739 011170 롯데케미칼 12월 결산 0.4 67.68
740 071840 롯데하이마트 12월 결산 -2.4 12.16
741 328130 루닛 12월 결산 -566.0 nan
742 038060 루멘스 12월 결산 2.6 10.0
743 162120 루켄테크놀러지스 12월 결산 -4.0 nan
744 253610 루트락 12월 결산 -43.2 nan
745 085370 루트로닉 12월 결산 61.9 8.45
746 060240 룽투코리아 12월 결산 -65.1 nan
747 058470 리노공업 12월 결산 70.6 15.73
748 039980 리노스 12월 결산 -5.6 nan
749 019570 리더스 기술투자 12월 결산 nan nan
750 016100 리더스코스메틱 12월 결산 0.5 22.09
751 328270 리독스바이오 12월 결산 2.0 7.17
752 012700 리드코프 12월 결산 9.1 3.85
753 302550 리메드 12월 결산 4.1 31.35
754 073570 리튬포어스 12월 결산 nan nan
755 377450 리파인 12월 결산 89.2 7.98
756 277070 린드먼아시아 12월 결산 nan nan
757 042500 링네트 12월 결산 16.7 5.47
758 219420 링크제니시스 12월 결산 59.5 16.81
759 439090 마녀공장 12월 결산 184.9 0.0
760 027740 마니커 12월 결산 3.6 101.59
761 195500 마니커에프앤지 12월 결산 -6.5 28.49
762 363170 마더스제약 12월 결산 -1.2 10.28
763 377480 마음AI 12월 결산 -48.6 nan
764 050480 마이다스IT 12월 결산 15.5 0.0
765 316640 마이지놈박스 12월 결산 -894.3 nan
766 305090 마이크로디지탈 12월 결산 -84

982 002580 삼미 12월 결산 -1.3 nan
983 012210 삼미금속 12월 결산 -2.1 90.88
984 053700 삼보모터스 12월 결산 4.1 4.61
985 009620 삼보산업 12월 결산 3.9 11.34
986 070080 삼보오토 12월 결산 5.7 nan
987 161990 삼보컴퓨터 12월 결산 -4.3 nan
988 023600 삼보판지 12월 결산 11.4 2.45
989 001470 삼부토건 12월 결산 -34.6 nan
990 006400 삼성SDI 12월 결산 16.9 13.41
991 006660 삼성공조 12월 결산 15.4 7.49
992 380320 삼성머스트스팩5호 12월 결산 nan nan
993 018360 삼성메디슨 12월 결산 38.3 nan
994 028260 삼성물산 12월 결산 41.2 8.45
995 207940 삼성바이오로직스 12월 결산 14.7 45.23
996 032830 삼성생명 12월 결산 nan nan
997 377630 삼성스팩4호 12월 결산 nan nan
998 425290 삼성스팩6호 12월 결산 nan nan
999 439250 삼성스팩7호 12월 결산 nan nan
1000 448740 삼성스팩8호 12월 결산 nan nan
1001 018260 삼성에스디에스 12월 결산 25.1 6.83
1002 028050 삼성엔지니어링 12월 결산 32.9 5.59
1003 009150 삼성전기 12월 결산 15.4 4.89
1004 005930 삼성전자 12월 결산 25.9 4.62
1005 001360 삼성제약 12월 결산 -39.3 nan
1006 010140 삼성중공업 12월 결산 -16.0 nan
1007 016360 삼성증권 12월 결산 nan nan
1008 068290 삼성출판사 12월 결산 -1.3 15.26
1009 029780 삼성카드 12월 결산 nan nan
1010 000810 삼성화재 12월 결산 nan nan
1011 006110 삼아알미늄 12월 결산 

...

1692 067900 와이엔텍 12월 결산 15.0 4.09
1693 007530 와이엠 12월 결산 4.0 10.02
1694 155650 와이엠씨 12월 결산 34.6 4.33
1695 273640 와이엠텍 12월 결산 57.6 13.59
1696 251370 와이엠티 12월 결산 1.8 22.51
1697 066430 와이오엠 12월 결산 1.4 42.87
1698 193250 와이제이엠게임즈 12월 결산 -9.7 232.49
1699 096250 와이즈넛 12월 결산 35.6 0.0
1700 273060 와이즈버즈 12월 결산 18.6 17.79
1701 354180 와이즈에이아이 월 결산 nan nan
1702 019210 와이지-원 12월 결산 6.0 5.45
1703 122870 와이지엔터테인먼트 12월 결산 15.6 16.67
1704 011690 와이투솔루션 12월 결산 3.9 24.81
1705 332570 와이팜 12월 결산 9.2 17.25
1706 079000 와토스코리아 12월 결산 0.6 26.96
1707 110030 완도전복 12월 결산 -7.7 nan
1708 070960 용평리조트 12월 결산 0.3 15.58
1709 316140 우리금융지주 12월 결산 nan nan
1710 032820 우리기술 12월 결산 16.6 23.15
1711 041190 우리기술투자 12월 결산 nan nan
1712 115440 우리넷 12월 결산 -13.3 nan
1713 046970 우리로 12월 결산 5.0 13.69
1714 082850 우리바이오 12월 결산 13.9 2.72
1715 298870 우리벤처파트너스 12월 결산 nan nan
1716 215360 우리산업 12월 결산 -15.6 nan
1717 072470 우리산업홀딩스 12월 결산 -6.9 100.48
1718 073560 우리손에프앤지 12월 결산 8.1 3.82
1719 037400 우리엔터프라이즈 12월 결산 10.6 3.07
1720 153490 우리이앤엘 12

1929 034590 인천도시가스 12월 결산 3.0 5.08
1930 211050 인카금융서비스 12월 결산 14.3 2.0
1931 083640 인콘 12월 결산 9.4 32.6
1932 216050 인크로스 12월 결산 55.3 9.3
1933 049070 인탑스 12월 결산 47.7 3.63
1934 119610 인터로조 12월 결산 14.2 9.77
1935 017250 인터엠 09월 결산 9.9 11.75
1936 129260 인터지스 12월 결산 12.1 3.02
1937 051370 인터플렉스 12월 결산 13.5 5.42
1938 064290 인텍플러스 12월 결산 25.6 9.13
1939 189300 인텔리안테크 12월 결산 7.6 24.83
1940 150840 인트로메딕 12월 결산 -44.1 nan
1941 048530 인트론바이오 12월 결산 -4.8 3165.98
1942 023810 인팩 12월 결산 8.2 5.19
1943 175140 인포마크 12월 결산 -91.7 nan
1944 115310 인포바인 12월 결산 62.5 8.76
1945 039290 인포뱅크 12월 결산 -3.6 86.02
1946 071200 인피니트헬스케어 12월 결산 -1.2 13.04
1947 101930 인화정공 12월 결산 8.8 13.99
1948 249420 일동제약 12월 결산 -26.9 nan
1949 000230 일동홀딩스 12월 결산 -26.4 nan
1950 013360 일성건설 12월 결산 9.2 22.25
1951 003120 일성신약 12월 결산 0.6 51.12
1952 333430 일승 12월 결산 5.2 29.14
1953 068330 일신바이오 12월 결산 19.7 12.27
1954 003200 일신방직 12월 결산 -4.1 35.09
1955 007110 일신석재 12월 결산 7.3 16.56
1956 007570 일양약품 12월 결산 11.2 8.54
1957 178780 일월지엠엘 12월 결산 -108.3 nan


2166 083550 케이엠 12월 결산 4.9 5.63
2167 032500 케이엠더블유 12월 결산 -19.8 nan
2168 225430 케이엠제약 12월 결산 -3.7 268.11
2169 102370 케이옥션 12월 결산 5.1 20.44
2170 381970 케이카 12월 결산 9.1 7.6
2171 036030 케이티알파 12월 결산 36.0 6.79
2172 064820 케이프 12월 결산 0.1 38.44
2173 256940 케이피에스 12월 결산 -36.5 nan
2174 024880 케이피에프 12월 결산 6.1 6.79
2175 042040 케이피엠테크 12월 결산 -1.8 60.75
2176 054410 케이피티유 12월 결산 13.6 20.72
2177 258610 케일럼 12월 결산 -4.0 nan
2178 274090 켄코아에어로스페이스 12월 결산 7.9 14.62
2179 402420 켈스 12월 결산 -16.4 nan
2180 089010 켐트로닉스 12월 결산 5.4 7.18
2181 220260 켐트로스 12월 결산 8.3 23.69
2182 176590 코나솔 12월 결산 7.0 19.93
2183 052400 코나아이 12월 결산 nan 4.34
2184 402030 코난테크놀로지 12월 결산 -1383.7 nan
2185 391710 코닉오토메이션 12월 결산 44.6 38.93
2186 046070 코다코 12월 결산 2.6 8.46
2187 047770 코데즈컴바인 12월 결산 347.6 13.13
2188 078940 코드네이처 12월 결산 -121.1 nan
2189 080530 코디 12월 결산 -12.6 nan
2190 104540 코렌텍 12월 결산 9.0 14.89
2191 040470 코리아2000 12월 결산 0.6 2.75
2192 027050 코리아나 12월 결산 1.2 25.52
2193 020920 코리아로터리 12월 결산 -17.8 nan
2194 007810 코리아써키트 12월 결산 14

2405 168360 펨트론 12월 결산 25.1 8.07
2406 087010 펩트론 12월 결산 -40.2 nan
2407 090080 평화산업 12월 결산 6.9 7.62
2408 010770 평화홀딩스 12월 결산 1.7 10.44
2409 119500 포메탈 12월 결산 3.7 10.4
2410 389140 포바이포 12월 결산 -49.2 nan
2411 022100 포스코DX 12월 결산 17.7 12.93
2412 058430 포스코스틸리온 12월 결산 7.8 3.98
2413 009520 포스코엠텍 12월 결산 6.9 35.51
2414 025100 포스코이앤씨 12월 결산 14.5 0.54
2415 047050 포스코인터내셔널 12월 결산 11.2 4.94
2416 003670 포스코퓨처엠 12월 결산 6.0 57.27
2417 189690 포시에스 06월 결산 15.1 8.97
2418 045020 포앤티 12월 결산 1.2 3.19
2419 318020 포인트모바일 12월 결산 6.9 34.48
2420 256630 포인트엔지니어링 12월 결산 -8.4 18.16
2421 331380 포커스에이치엔에스 12월 결산 10.8 13.87
2422 234100 폴라리스세원 12월 결산 6.1 11.44
2423 041020 폴라리스오피스 12월 결산 18.1 23.78
2424 114630 폴라리스우노 12월 결산 17.1 5.59
2425 290720 푸드나무 12월 결산 -5.6 262.34
2426 005670 푸드웰 12월 결산 4.6 9.34
2427 094940 푸른기술 12월 결산 7.9 24.74
2428 007330 푸른저축은행 12월 결산 nan nan
2429 017810 풀무원 12월 결산 4.1 8.21
2430 093380 풍강 08월 결산 3.8 6.32
2431 023900 풍국주정 12월 결산 2.9 14.36
2432 103140 풍산 12월 결산 7.2 5.41
2433 005810 풍산홀딩스 12월 결산 25.

2641 009830 한화솔루션 12월 결산 6.2 8.64
2642 272210 한화시스템 12월 결산 2.2 15.6
2643 012450 한화에어로스페이스 12월 결산 7.2 8.02
2644 042660 한화오션 12월 결산 -43.8 nan
2645 003530 한화투자증권 12월 결산 nan nan
2646 386580 한화플러스제2호스팩 12월 결산 nan nan
2647 430460 한화플러스제3호스팩 12월 결산 nan nan
2648 195870 해성디에스 12월 결산 41.6 2.5
2649 034810 해성산업 12월 결산 10.0 4.54
2650 076610 해성옵틱스 12월 결산 -23.1 88.47
2651 059270 해성티피씨 12월 결산 -5.2 nan
2652 058480 해원에스티 12월 결산 0.3 33.03
2653 101530 해태제과식품 12월 결산 -2.2 7.97
2654 220180 핸디소프트 12월 결산 -2.4 33.71
2655 143210 핸즈코퍼레이션 12월 결산 -8.2 nan
2656 900270 헝셩그룹 12월 결산 59.8 3.82
2657 214180 헥토이노베이션 12월 결산 483.2 6.1
2658 234340 헥토파이낸셜 12월 결산 nan 11.19
2659 084990 헬릭스미스 12월 결산 -55.3 nan
2660 000720 현대건설 12월 결산 8.6 8.17
2661 170030 현대공업 12월 결산 23.4 4.96
2662 453340 현대그린푸드 12월 결산 nan nan
2663 086280 현대글로비스 12월 결산 23.1 2.86
2664 064350 현대로템 12월 결산 18.0 17.84
2665 079430 현대리바트 12월 결산 -3.8 22.79
2666 012330 현대모비스 12월 결산 12.0 6.47
2667 319400 현대무벡스 12월 결산 8.8 22.46
2668 010620 현대미포조선 12월 결산 -4.3 nan
2669 048410 현